In [4]:
# Cell 1: Imports and setup
import sys
from pathlib import Path

# Adding src to path for imports
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.append(str(project_root / 'src'))

# Custom modules
from churn_analysis.config.settings import TELCO_RAW_FILE, FIGURE_SIZE, RANDOM_STATE

# Standard libraries
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from rich import print

# Notebook setup
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = FIGURE_SIZE
sns.set_palette("husl")


In [7]:
# Cell 2: Load and initial inspection
from churn_analysis.data.loaders.loaders import load_telco_data, get_dataset_info

print("🔍 Loading Telco Customer Churn Dataset...")
df= load_telco_data()

info= get_dataset_info(df)
print(f"📊 Dataset Info: {info}")

🔍 Loading Telco Customer Churn Dataset...

✅ Dataset loaded: 7043 rows, 21 columns


📊 Dataset Info: {'shape': (7043, 21), 'columns': ['customerID', 'gender', 'SeniorCitizen', 'Partner', 
'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 
'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'], 'null_count': 11, 'dtypes': {'customerID': 'String', 
'gender': 'String', 'SeniorCitizen': 'Int64', 'Partner': 'String', 'Dependents': 'String', 'tenure': 'Int64', 
'PhoneService': 'String', 'MultipleLines': 'String', 'InternetService': 'String', 'OnlineSecurity': 'String', 
'OnlineBackup': 'String', 'DeviceProtection': 'String', 'TechSupport': 'String', 'StreamingTV': 'String', 
'StreamingMovies': 'String', 'Contract': 'String', 'PaperlessBilling': 'String', 'PaymentMethod': 'String', 
'MonthlyCharges': 'Float64', 'TotalCharges': 'Float64', 'Churn': 'String'}}